# Exercício 8 - Prompt Engineering - LLaMA 3 (70B) Groq

### Aluno: Pedro Rodrigues Corrêa
### RA: 243236

## Parameters

In [2]:
NUM_TEST_SAMPLES = 1000
MODEL_NAME = "llama3-70b-8192"

## Install and Import

In [3]:
!pip install -q datasets
!pip install -q torch
!pip install -q groq
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 10.2 MB/s eta 0:00:00


In [4]:
from collections import Counter
from datasets import load_dataset
import re
import torch
import os
from transformers import LlamaTokenizerFast
import pandas as pd
import getpass

## Groq

In [5]:
'''def load_grok_key():
    try:
        # Open and read the entire content of the file
        with open("groq_key.txt", 'r') as file:
            contents = file.read()

        return contents

    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None

grok_key = load_grok_key()
os.environ["GROQ_API_KEY"] = grok_key'''

In [9]:
grok_key = getpass.getpass("Groq api key:")

groq api key:··········


In [25]:
from groq import Groq

client = Groq(api_key=grok_key)

def groq_chat(content):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

## Dataset

In [129]:
# Load the IMDb movie reviews dataset
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

#test_dataset = test_dataset.shuffle(seed=42).select(range(10))

In [132]:
indexes = list(range(8797, 9260)) + list(range(21297, 21760))
test_dataset = test_dataset.select(indexes)


In [133]:
train_labels = train_dataset['label']
item_counts = Counter(train_labels)
print(f'Train Dataset Label breakdown: {item_counts}')

tst_labels = test_dataset['label']
item_counts = Counter(tst_labels)
print(f'Test Dataset Label breakdown: {item_counts}')

Train Dataset Label breakdown: Counter({0: 12500, 1: 12500})
Test Dataset Label breakdown: Counter({0: 463, 1: 463})


## Data Preparation

In [118]:
template_zero = """Your task is to classify sentences' sentiment as 'positive' or 'negative'. Your answer should be one word, either 'positive' or 'negative'.
Sentence:{text}.
Answer:"""

In [119]:
template_few = """Your task is to classify sentences' sentiment as 'positive' or 'negative'. Your answer should be one word, either 'positive' or 'negative'.

Example 1:
Sentence: Incredible acting and stunning visuals. Truly loved it!
Answer:'positive'

Example 2:
Sentence: Boring plot and predictable ending. Not impressed.
Answer:'negative'

Example 3:
Sentence: Best movie this year, totally heartwarming and exciting!
Answer:'positive'

Example 4:
Sentence: Terrible pacing and dull characters. Waste of time.
Answer:'negative'

Now it's your turn:
Sentence:{text}.
Answer:"""

In [120]:
template_cot = """Your task is to classify sentences' sentiment as 'positive' or 'negative'. Your answer should be one word, either 'positive' or 'negative'. Here's how you should think about it:
- First, consider the overall emotion conveyed by the sentence. Words like 'love', 'happy', 'enjoy' indicate a positive sentiment, while words like 'hate', 'sad', 'disappoint' suggest a negative sentiment.
- Consider the context of the sentence to ensure the sentiment isn't negated by words like 'not' or 'never'.
- If the sentence contains primarily positive terms and no significant negation, it is positive. If it contains negative terms or is negated, it is negative.
Sentence:{text}.
Answer:"""


In [134]:
POSITIVE_LABEL = "positive"
NEGATIVE_LABEL = "negative"

def preprocess(example):
    pattern = re.compile(r'\s+<.*?>')

    text  = example["text"]
    label = example["label"]

    text = re.sub(pattern, ' ', text)
    return {
        "text": text.strip(),
        "class": POSITIVE_LABEL if label == 1 else NEGATIVE_LABEL
    }

def mapper(data, template):
    test_dataset = data.map(lambda example: {'text': example['text'].replace("<br />", " ")})
    test_dataset = test_dataset.map(lambda example: {"text": template.format(**example)})
    test_dataset = test_dataset.map(lambda example: {"class": POSITIVE_LABEL if example["label"] == 1 else NEGATIVE_LABEL})

    return test_dataset

In [135]:
test_zero = mapper(test_dataset, template_zero)
test_few = mapper(test_dataset, template_few)
test_cot = mapper(test_dataset, template_cot)

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

In [136]:
test_zero[0]

{'text': 'Your task is to classify sentences\' sentiment as \'positive\' or \'negative\'. Your answer should be one word, either \'positive\' or \'negative\'.\nSentence:MASTER PLAN: have the winning team in a deadly tournament. One of several martial arts action pictures that attempted to capture the flavor of the famous "Enter the Dragon" from \'73, this one is an effort from South Africa. The villain\'s stronghold is a bit different, appearing as a white castle-like fortress in the middle of the desert from a distance. The villain himself, a Baron or general, is a slightly more perverse version of the "Dr.No" or Han mold of master villainy, having strange flashbacks to the glory days of Nazi Germany. He does wear the full regalia Nazi uniform at some points. His main ambition in life is to hold an illegal martial arts competition/tournament against his Japanese rival, an extension of their complicity in the 2nd World War (my army is better than your army). It sounds silly and it is, 

In [137]:
output = groq_chat(test_zero['text'][0])
print(output)

Negative


## Avaliação

In [127]:
from tqdm import tqdm

def eval_model(data):
    predictions = []

    labels = data["class"]
    reviews = data["text"]

    for review in tqdm(reviews):
        predicted = groq_chat(review).lower()
        predictions.append(predicted)

    correct = sum([1 for p, r in zip(predictions, labels) if p.lower() == r.lower()])
    total = len(predictions)
    acc = correct/total

    print(f'Model Accuracy = {acc*100}%')

  #return predictions_raw, predictions

In [139]:
# Avaliação Zero-Shot

print("Avaliando Modelo Zero-Shot")
eval_model(test_zero)

Avaliando Modelo Zero-Shot


100%|██████████| 926/926 [53:06<00:00,  3.44s/it]

Model Accuracy = 94.9244060475162%


In [ ]:
# Avaliação Few-Shot

print("Avaliando Modelo Few-Shot")
eval_model(test_few)

Avaliando Modelo Few-Shot


 15%|█▍        | 138/926 [11:36<33:25,  2.55s/it]

In [ ]:
# Avaliação Chain-Of-Thought

print("Avaliando Modelo Chain-Of-Thought")
eval_model(test_cot)